In [1]:

import sys
sys.path.append('..')
sys.path.append('../ehrshot')
import argparse
import json
import os
from typing import Any, Dict, List, Optional, Tuple, Union
import numpy as np
import collections
import pandas as pd
import sklearn
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from loguru import logger
from sklearn.preprocessing import MaxAbsScaler
from utils import (
    LABELING_FUNCTION_2_PAPER_NAME,
    SHOT_STRATS,
    MODEL_2_INFO,
    get_labels_and_features, 
    process_chexpert_labels, 
    convert_multiclass_to_binary_labels,
    CHEXPERT_LABELS, 
    LR_PARAMS, 
    XGB_PARAMS, 
    RF_PARAMS,
    ProtoNetCLMBRClassifier, 
    get_patient_splits_by_idx
)
from sklearn.model_selection import GridSearchCV, PredefinedSplit
from scipy.sparse import issparse
import scipy
import lightgbm as lgb
import femr
import femr.datasets
from femr.labelers import load_labeled_patients, LabeledPatients

import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score
from tqdm import tqdm
torch.manual_seed(42)

In [2]:
class TwoLayerNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(TwoLayerNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [3]:
# def train_model(num_epochs, model, train_loader, val_loader, criterion, optimizer):
#     for epoch in range(num_epochs):
#         model.train()
#         for inputs, labels in train_loader:
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#         # Evaluate on validation set
#         if epoch % 2 == 0:
#             model.eval()
#             with torch.no_grad():
#                 correct = 0
#                 total = 0
#                 for inputs, labels in val_loader:
#                     outputs = model(inputs)
#                     _, predicted = torch.max(outputs.data, 1)
#                     total += labels.size(0)
#                     correct += (predicted == labels).sum().item()
#                 val_accuracy = 100 * correct / total
#                 if epoch % 10 == 0:
#                     pass
#                     print(f'Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {val_accuracy:.2f}%')
#     return model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def train_model(num_epochs, model, train_loader, val_loader, criterion, optimizer, task_name):
    val_accuracy_current = 0
    val_auc = 0
    for epoch in range(num_epochs):
        print(f'Epoch [{epoch+1}/{num_epochs}]')
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            # print(outputs.device.type, labels.device.type)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Evaluate on validation set
        if epoch % 2 == 0:
            model.eval()
            with torch.no_grad():
                preds = []
                gts = []
                correct = 0
                total = 0
                
                for inputs, labels in val_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    _, predicted = torch.max(outputs.data, 1)
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                    preds.extend(list(predicted.cpu().numpy()))
                    gts.extend(list(labels.cpu().numpy()))
                val_accuracy = 100 * correct / total
                auc_score = roc_auc_score(gts, preds)
                
                
                if auc_score > val_auc:
                    val_auc = auc_score
                    model_name = f'single_task_models/best_model_task_{task_name}.pth'
                    torch.save(model.state_dict(), model_name)
                print(f'Epoch [{epoch+1}/{num_epochs}], Validation Auc: {auc_score:.2f}%')
    return model, model_name

In [4]:
# def evaluate_model(model, test_loader):
#     model.eval()
#     preds = []
#     gts = []
#     with torch.no_grad():
#         correct = 0
#         total = 0
#         for inputs, labels in test_loader:
#             outputs = model(inputs)
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()
#             preds.extend(list(predicted.numpy()))
#             gts.extend(list(labels.numpy()))
#         test_accuracy = 100 * correct / total
#         print(f'Test Accuracy: {test_accuracy:.2f}%')
#         auc_score = roc_auc_score(gts, preds)
#     return test_accuracy, auc_score

def evaluate_model(model, test_loader):
    model.eval()
    preds = []
    preds_prob = []
    probs = []
    gts = []
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            # _, predicted = torch.max(outputs.data, 1)

            probabilities = torch.softmax(outputs.data, dim=1)

            # Get the predicted class for each data point (highest probability)
            predicted = torch.argmax(probabilities, dim=1)

            # Gather the probabilities of the predicted classes
            predicted_probabilities = probabilities[torch.arange(probabilities.shape[0]), predicted]

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            preds.extend(list(predicted.cpu().numpy()))
            gts.extend(list(labels.cpu().numpy()))
            preds_prob.extend(list(predicted_probabilities.cpu().numpy()))
            probs.extend(list(probabilities.cpu().numpy()))

        test_accuracy = 100 * correct / total
        print(f'Test Accuracy: {test_accuracy:.2f}%')
        auc_score = roc_auc_score(gts, preds)
        ave_preds_prob = np.mean(preds_prob)

    return test_accuracy, auc_score * 100, ave_preds_prob, gts, preds, preds_prob, probs

In [5]:
labeling_functions=[
    "guo_los",
    "guo_readmission",
    "guo_icu",
    "new_hypertension",
    "new_hyperlipidemia",
    "new_pancan",
    "new_celiac",
    "new_lupus",
    "new_acutemi",
    "lab_thrombocytopenia",
    "lab_hyperkalemia",
    "lab_hyponatremia",
    "lab_anemia",
    "lab_hypoglycemia" # will OOM at 200G on `gpu` partition
]

In [6]:
path_to_database='../EHRSHOT_ASSETS/femr/extract'
path_to_labels_dir='../EHRSHOT_ASSETS/benchmark'
path_to_features_dir='../EHRSHOT_ASSETS/features'
path_to_output_dir='../uncertainty_quantification/single_task_results'
path_to_output_data_dir = '../uncertainty_quantification/single_task_data'
path_to_split_csv='../EHRSHOT_ASSETS/splits/person_id_map.csv'
path_to_data_csv = '../uncertainty_quantification'


In [7]:
# def parse_args() -> argparse.Namespace:
parser = argparse.ArgumentParser(description="Run EHRSHOT evaluation benchmark on a specific task.")
parser.add_argument("--path_to_database", required=True, type=str, help="Path to FEMR patient database")
parser.add_argument("--path_to_labels_dir", required=True, type=str, help="Path to directory containing saved labels")
parser.add_argument("--path_to_features_dir", required=True, type=str, help="Path to directory containing saved features")
parser.add_argument("--path_to_output_dir", required=True, type=str, help="Path to directory where results will be saved")
parser.add_argument("--path_to_split_csv", required=True, type=str, help="Path to CSV of splits")
parser.add_argument("--labeling_function", required=True, type=str, help="Labeling function for which we will create k-shot samples.", choices=LABELING_FUNCTION_2_PAPER_NAME.keys(), )
parser.add_argument("--num_threads", type=int, help="Number of threads to use")
parser.add_argument("--is_force_refresh", action='store_true', default=False, help="If set, then overwrite all outputs")
# return parser.parse_args()

_StoreTrueAction(option_strings=['--is_force_refresh'], dest='is_force_refresh', nargs=0, const=True, default=False, type=None, choices=None, required=False, help='If set, then overwrite all outputs', metavar=None)

In [8]:
acc_list = []
auc_list = []
uq_list = []
task_list = []
prob_list = []

# for i in tqdm(range(10, len(labeling_functions))):
# for i in tqdm(range(0, 5)):
# for i in tqdm(range(5, 10)):
# for i in [0]:
for i in range(len(labeling_functions)):
    labeling_function = labeling_functions[i]
    args = parser.parse_args(f'--labeling_function {labeling_function} --path_to_database {path_to_database} --path_to_labels_dir {path_to_labels_dir} --path_to_features_dir {path_to_features_dir} --path_to_split_csv {path_to_split_csv} --path_to_output_dir {path_to_output_dir}'.split())

    LABELING_FUNCTION: str = args.labeling_function
    # SHOT_STRAT: str = args.shot_strat
    # NUM_THREADS: int = args.num_threads
    IS_FORCE_REFRESH: bool = args.is_force_refresh
    PATH_TO_DATABASE: str = args.path_to_database
    PATH_TO_FEATURES_DIR: str = args.path_to_features_dir
    PATH_TO_LABELS_DIR: str = args.path_to_labels_dir
    PATH_TO_SPLIT_CSV: str = args.path_to_split_csv
    PATH_TO_LABELED_PATIENTS: str = os.path.join(PATH_TO_LABELS_DIR, LABELING_FUNCTION, 'labeled_patients.csv')
    PATH_TO_OUTPUT_DIR: str = args.path_to_output_dir
    PATH_TO_OUTPUT_FILE: str = os.path.join(PATH_TO_OUTPUT_DIR, f'{LABELING_FUNCTION}_results.csv')
    # PATH_TO_OUTPUT_DATA: str = 

    database = femr.datasets.PatientDatabase('../EHRSHOT_ASSETS/femr/extract')

    labeled_patients: LabeledPatients = load_labeled_patients(PATH_TO_LABELED_PATIENTS)
    patient_ids, label_values, label_times, feature_matrixes = get_labels_and_features(labeled_patients, PATH_TO_FEATURES_DIR)
    train_pids_idx, val_pids_idx, test_pids_idx = get_patient_splits_by_idx(PATH_TO_SPLIT_CSV, patient_ids)

    if LABELING_FUNCTION == "chexpert":
        label_values = process_chexpert_labels(label_values)
        sub_tasks: List[str] = CHEXPERT_LABELS
    elif LABELING_FUNCTION.startswith('lab_'):
        # Lab value is multi-class, convert to binary
        label_values = convert_multiclass_to_binary_labels(label_values, threshold=1)
        sub_tasks: List[str] = [LABELING_FUNCTION]
    else:
        # Binary classification
        sub_tasks: List[str] = [LABELING_FUNCTION]
            

    model = 'clmbr'
    X_train: np.ndarray = feature_matrixes[model][train_pids_idx]
    X_val: np.ndarray = feature_matrixes[model][val_pids_idx]
    X_test: np.ndarray = feature_matrixes[model][test_pids_idx]

    y_train: np.array = label_values[train_pids_idx].astype(int)
    y_val: np.array = label_values[val_pids_idx].astype(int)
    y_test: np.ndarray = label_values[test_pids_idx].astype(int)

    class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weights = torch.tensor(class_weights, dtype=torch.float)

    # Move weights to the same device as model
    if torch.cuda.is_available():
        class_weights = class_weights.to(device)

    # Convert numpy arrays to PyTorch tensors
    X_train = torch.tensor(X_train).float()
    X_val = torch.tensor(X_val).float()
    X_test = torch.tensor(X_test).float()
    y_train = torch.tensor(y_train).long()
    y_val = torch.tensor(y_val).long()
    y_test = torch.tensor(y_test).long()

    # Create TensorDatasets
    train_dataset = 
    
    set(X_train, y_train)
    val_dataset = TensorDataset(X_val, y_val)
    test_dataset = TensorDataset(X_test, y_test)

    # Create DataLoaders
    batch_size = 64  # You can adjust this size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # Assuming y_train is your label array for the training set

    input_size = X_train.shape[1]
    hidden_size = 100  # You can tune this
    num_classes = len(torch.unique(y_train))
    model = TwoLayerNet(input_size, hidden_size, num_classes)
    model.to(device)
    # Loss and optimizer
    criterion = nn.CrossEntropyLoss(weight=class_weights)  # Suitable for classification with imbalanced dataset
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)  # Learning rate can be tuned

    
    model, model_name = train_model(50, model, train_loader, val_loader, criterion, optimizer, task_name=labeling_function)
    
    del model
    best_model = TwoLayerNet(input_size, hidden_size, num_classes)
    best_model.load_state_dict(torch.load(model_name))
    best_model.to(device)
    
    acc, auc, uq, gts, preds, preds_prob, probs = evaluate_model(best_model, test_loader)


    pd.DataFrame([gts, preds, preds_prob, np.array(probs)]).T.to_csv(f'{path_to_output_dir}/{labeling_function}_results.csv', index=False)

    acc_list.append(acc)
    auc_list.append(auc)
    uq_list.append(uq)
    task_list.append(labeling_function)
    prob_list.append(probs)

Epoch [1/50]
Epoch [1/50], Validation Auc: 0.72%
Epoch [2/50]
Epoch [3/50]
Epoch [3/50], Validation Auc: 0.74%
Epoch [4/50]
Epoch [5/50]
Epoch [5/50], Validation Auc: 0.69%
Epoch [6/50]
Epoch [7/50]
Epoch [7/50], Validation Auc: 0.71%
Epoch [8/50]
Epoch [9/50]
Epoch [9/50], Validation Auc: 0.71%
Epoch [10/50]
Epoch [11/50]
Epoch [11/50], Validation Auc: 0.70%
Epoch [12/50]
Epoch [13/50]
Epoch [13/50], Validation Auc: 0.70%
Epoch [14/50]
Epoch [15/50]
Epoch [15/50], Validation Auc: 0.69%
Epoch [16/50]
Epoch [17/50]
Epoch [17/50], Validation Auc: 0.68%
Epoch [18/50]
Epoch [19/50]
Epoch [19/50], Validation Auc: 0.68%
Epoch [20/50]
Epoch [21/50]
Epoch [21/50], Validation Auc: 0.69%
Epoch [22/50]
Epoch [23/50]
Epoch [23/50], Validation Auc: 0.69%
Epoch [24/50]
Epoch [25/50]
Epoch [25/50], Validation Auc: 0.69%
Epoch [26/50]
Epoch [27/50]
Epoch [27/50], Validation Auc: 0.68%
Epoch [28/50]
Epoch [29/50]
Epoch [29/50], Validation Auc: 0.69%
Epoch [30/50]
Epoch [31/50]
Epoch [31/50], Validation

In [9]:
# next(model.parameters()).device
path_to_output_data_dir


'../uncertainty_quantification/single_task_data'

In [10]:
df_temp

NameError: name 'df_temp' is not defined